# 파일 경로 지정 및 라이브러리 실행

In [44]:
import pandas as pd
import numpy as np
import xlwings as xw

In [75]:
# Backdata(Dataframe)
financial_data_path = r"https://raw.githubusercontent.com/DongwooChae/QuantifyPro/master/Udemy_Python_Data_science_PANDAS/MTP_valuation/241Q%EC%9E%AC%EB%AC%B4%EB%8D%B0%EC%9D%B4%ED%84%B0.csv"
stock_count_path = r"https://raw.githubusercontent.com/DongwooChae/QuantifyPro/master/Udemy_Python_Data_science_PANDAS/MTP_valuation/241Q%EC%A3%BC%EC%8B%9D%EC%88%98.csv"
main_products_path = r"C:\Users\User\Documents\GitHub\QuantifyPro\Udemy_Python_Data_science_PANDAS\MTP_valuation\주요상품.csv"
stockprice2_path = r"C:\Users\User\Documents\GitHub\QuantifyPro\Udemy_Python_Data_science_PANDAS\MTP_valuation\stockprice_0722.csv"
industry_path = r"https://raw.githubusercontent.com/DongwooChae/QuantifyPro/master/Udemy_Python_Data_science_PANDAS/MTP_valuation/%ED%91%9C%EC%A4%80%EC%82%B0%EC%97%85%EB%B6%84%EB%A5%98.csv"

In [74]:
# 엑셀 파일 경로 및 파일 열기

file_path = r'C:\Users\User\Documents\GitHub\QuantifyPro\Udemy_Python_Data_science_PANDAS\MTP_valuation\multiple_valuation.xlsx'
wb = xw.Book(file_path)  # 엑셀 파일 열기
input_sheet = wb.sheets['input']

# 필요한 인풋 파라미터 정리
- 평가기준일
- 평가대상기업의 표준산업분류, 주요사업, 주요상품 >> 유사기업 선정의 근거
- main_products 데이터프레임에 영업정지, 관리여부 추가할 것

# 데이터 불러오기 및 주요 데이터 병합

In [47]:
stockprice2 = pd.read_csv(stockprice2_path, index_col ="company")
# 열 이름을 'Timestamp' 형식으로 변환
stockprice2.columns = pd.to_datetime(stockprice2.columns)


In [48]:
financial_data = pd.read_csv(financial_data_path, index_col="company")
stock_count = pd.read_csv(stock_count_path, index_col="company")
main_products = pd.read_csv(main_products_path, index_col="company")

In [49]:
# 인덱스 기준으로 병합 (company 인덱스를 기준으로 stock_count와 main_products 병합)
# '10차표준(세분류)', '주요상품', '영업현황'을 결합
merged_data = stock_count.join(main_products[['10차표준(세분류)', '주요상품', '영업현황']], how='left')


In [69]:
# merged_data.head()

## 유통보통주식수, EPS, BPS, SPS 계산은 불필요(이미 완료되어 있음)

### stock_count 데이터프레임에 주식수부터 EPS, BPS, SPS가 입력되어 있으므로 financial_data 데이터프레임은 불필요

In [70]:
# stock_count.head(50)

In [52]:
# 엑셀에서 평가기준일, 배수종료일, 배수시작일 가져오기
evaluation_date = input_sheet.range('B3').value  # 평가기준일: 2024-06-30
end_date = input_sheet.range('B4').value  # 배수종료일: 2024-04-30
start_date = input_sheet.range('B5').value  # 배수시작일: 2023-10-31

# 배수시작일부터 배수종료일까지 주가 데이터를 필터링
filtered_data = stockprice2.loc[:, start_date:end_date]


In [53]:
print(f"성공적으로 날짜를 불러왔습니다. 평가기준일:{evaluation_date}, 배수종료일:{end_date}, 배수시작일{start_date}")

성공적으로 날짜를 불러왔습니다. 평가기준일:2024-06-30 00:00:00, 배수종료일:2024-04-30 00:00:00, 배수시작일2023-10-31 00:00:00


In [54]:
# 날짜 열이 존재하는 경우에만 필터링 (날짜가 존재하는 열만 선택)
existing_dates = stockprice2.columns[(stockprice2.columns >= start_date) & (stockprice2.columns <= end_date)]

# 해당 기간 동안의 주가 데이터를 필터링
filtered_data = stockprice2.loc[:, existing_dates]

# 해당 기간 동안의 주가 평균 계산 (각 회사별 평균)
average_prices_subset = filtered_data.mean(axis=1)

# stock_count 데이터프레임에 주가 평균값을 맨 마지막 열로 추가
stock_count['average_price'] = average_prices_subset


In [55]:
# PER, PBR, PSR 계산
# average_price를 분자로 하고, eps, bps, sps를 각각 분모로 사용

# PER 계산: 주가 / EPS
stock_count['PER'] = stock_count['average_price'] / stock_count['eps']

# PBR 계산: 주가 / BPS
stock_count['PBR'] = stock_count['average_price'] / stock_count['bps']

# PSR 계산: 주가 / SPS
stock_count['PSR'] = stock_count['average_price'] / stock_count['sps']

# 유사기업선정 - 표준산업분류

In [56]:
# 2. 엑셀에서 B8 ~ B32 셀의 값을 리스트로 가져오고, None 값을 제외
keywords = [keyword for keyword in input_sheet.range('B8:B32').value if keyword is not None]

In [57]:
# 3. main_product 데이터프레임 불러오기
main_products_path = r'C:\Users\User\Documents\GitHub\QuantifyPro\Udemy_Python_Data_science_PANDAS\MTP_valuation\주요상품.csv'
main_products = pd.read_csv(main_products_path, index_col="company")

In [58]:
# 4. B8 ~ B32 셀의 키워드가 '10차표준(세분류)' 열에 포함되는지를 확인하는 OR 조건 생성
# 여러 단어를 OR 조건으로 결합하여 str.contains()에 전달
pattern = '|'.join(keywords)

In [59]:
# 5. 불리언 마스킹
mask = main_products['10차표준(세분류)'].str.contains(pattern, case=False, na=False)

In [60]:
filtered_index = main_products[mask].index

In [61]:
filtered_stockprice = stockprice2.loc[filtered_index.intersection(stockprice2.index)]

In [62]:
backdata_sheet = wb.sheets['backdata']  # 출력할 시트 지정
# 필터링된 데이터 출력 (엑셀의 backdata 시트에 A1 셀부터 출력)
backdata_sheet.range('A1').value = filtered_stockprice

In [63]:
# 6. 조건을 만족하는 데이터 필터링
filtered_data = main_products[mask]
number_of_company = filtered_data.shape[0]
print(f"표준산업분류 조건을 만족하는 상장기업의 수는 {number_of_company}개 입니다.")

표준산업분류 조건을 만족하는 상장기업의 수는 26개 입니다.


In [64]:
valid_index = filtered_data.index.intersection(merged_data.index)
company_list = merged_data.loc[valid_index]

In [65]:
# PER 계산: 주가 / EPS
company_list['PER'] = stock_count['average_price'] / stock_count['eps']

# PBR 계산: 주가 / BPS
company_list['PBR'] = stock_count['average_price'] / stock_count['bps']

# PSR 계산: 주가 / SPS
company_list['PSR'] = stock_count['average_price'] / stock_count['sps']

In [66]:
# 출력할 데이터 준비 (인덱스, 10차표준(세분류), PER, PBR, PSR)
output_data = company_list[['10차표준(세분류)', 'PER', 'PBR', 'PSR']]
output_data.insert(0, 'Company', company_list.index)  # 인덱스를 'Company' 열로 추가

In [67]:
multiple = company_list[['PER','PBR','PSR']].mean()
output_sheet = wb.sheets['output']  # 출력할 시트

# multiple 값 출력 (B32, B33, B34)
output_sheet.range('B32').value = multiple['PER']
output_sheet.range('B33').value = multiple['PBR']
output_sheet.range('B34').value = multiple['PSR']

In [68]:
# 엑셀에 데이터 출력 (A2부터 시작)
output_sheet.range('A2').value = output_data.values  # 데이터 값 출력
output_sheet.range('A1').value = output_data.columns.tolist()  # 헤더 출력

### ▲▲▲▲▲ 여기까지 작업 완료 ▲▲▲▲▲